In [ ]:
#rodar so uma vez
# instala o slenium 
!pip install selenium

In [ ]:
# Chrome -> chromedriver
# firefox -> geckodriver

# baixar o arquivo do webdriver e botar na mesma pasta do arquivo que vai rodar

from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

# abrir um navegador
navegador = webdriver.Chrome("chromedriver.exe") # ( nome do arquivo que vc baixou antes )

navegador.get("https://www.google.com/") # pagina que vai ser aberta no navegador 

# Passo 1: Pegar a cotação do Dólar
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar") # pesquisa
    # como achar o conteudo do (1°) vai na pagina botão direito INSPECIONAR aperta na [->] apertar onde vc quer pra marcar o codigo 
    # botão direito na parte marcada COPIAR escolher COPIAR XPATH
    # usar '' simples colar o 'xpath'
    # send_keys ( o que vc vai buscar )

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER) # enter de busca

cotacao_dolar = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") # pega o valor 

print(cotacao_dolar)

# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")

cotacao_ouro = cotacao_ouro.replace(",", ".") # tarta caso o numero venha com , e trocando ele por .

print(cotacao_ouro)

navegador.quit()

In [ ]:
# Passo 4: Importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

In [ ]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
# filtra as linhas [tabela[" a coluna que vc quer achar as linhas"] == "o conteudo das linhas que vc quer mudar", "coluna que vc quer editar"]
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

In [ ]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Novo.xlsx", index=False) # cria um novo arquivo mas se usar o mesmo nome do original ele substitui
# index=False remove a coluna que tem os índices

navegador.quit() # fecha o navegador quando termina